In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import os.path
import sys
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
import scoping
from scoping.models import *
from utils.text import *

In [2]:
df = pd.read_csv('data/documents_parsed.csv')
print(df.shape)
df = df[pd.notna(df['redirect_url'])]
print(df.shape)
df.head()

(385, 6)
(337, 6)


,wosarticle__ti,wosarticle__py,wosarticle__doc,wosarticle__di,docproject__relevant,redirect_url
0,The European Commission on Sustainable Develop...,NaN,5213276,http://dx.doi.org/10.1080/07360932.2022.2032255,Yes (1),https://www.tandfonline.com/doi/full/10.1080/0...
4,A bottom-up dynamic building stock model for r...,2022.0,4915915,http://dx.doi.org/10.1016/j.apenergy.2021.118060,Yes (1),https://linkinghub.elsevier.com/retrieve/pii/S...
5,Destination net-zero: what does the internatio...,2022.0,4916232,http://dx.doi.org/10.1080/09669582.2021.1962890,Yes (1),https://www.tandfonline.com/doi/full/10.1080/0...
6,Possibilities of Ammonia as Both Fuel and NOx ...,2022.0,4947789,http://dx.doi.org/10.3390/jmse10010043,Yes (1),https://www.mdpi.com/2077-1312/10/1/43/htm
7,Potential effects of the EU's carbon border ad...,2022.0,5196097,http://dx.doi.org/10.1007/s10668-021-01779-1,Yes (1),https://link.springer.com/article/10.1007/s106...


In [21]:
def get_ps(html_doc):  
    soup = BeautifulSoup(html_doc, 'html.parser')
    # MDPI
    body = soup.find("div", class_="html-body")
    if body is not None:
        ps = body.find_all("div", class_="html-p")
        if len(ps) > 0:
            return ps
        else:
            print(body.prettify())
            return []
    # Springer nature
    body = soup.find("div", attrs={"data-article-body": "true"})
    if body is not None:
        ps = body.find_all("p")
        if len(ps) > 0:
            return ps
        else:
            print(body.prettify())   
            return []
    # T&F
    body = soup.find("div", class_="hlFld-Fulltext")
    if body is not None:
        ps = body.find_all("p")
        if len(ps) > 0:
            return ps
        else:
            print(body.prettify())
            return []
    
    print(soup.prettify())
    return []

q = Query.objects.get(pk=10704)

tag, created = Tag.objects.get_or_create(
    query = q,
    title = "Paragraph review"
)
      
users = User.objects.filter(projectroles__project=p)
    
for i, row in df.reset_index().iterrows():
    fname = row['wosarticle__di'].split('org/')[-1].replace('/','__').replace('.','_')
    print(fname)
    print(row['redirect_url'])
    try:
        with open(f"fulltexts/{fname}.html") as f:
            ps = get_ps(f)
            print(len(ps))
    except:
        print("could not find file ?")
        continue
        
    doc = Doc.objects.get(pk=row['wosarticle__doc'])
    if not doc.docpar_set.exists():
        print("Creating new docpar objects")
        for i, p in enumerate(ps):
            dp, created = DocPar.objects.get_or_create(
                doc = doc,
                text = p.get_text(),
                n = i
            )
            
    for u in users:
        for dp in doc.docpar_set.all():
            do, created = DocOwnership.objects.get_or_create(
                docpar = dp, user = u, tag = tag, query = q
            )
        
    break
        
    tag.update_tag()
    if len(ps) < 2:
        break
        
    

10_1080__07360932_2022_2032255
https://www.tandfonline.com/doi/full/10.1080/07360932.2022.2032255
35


In [20]:
django.db.connection.close()
p = q.project
users = User.objects.filter(projectroles__project=p)

<QuerySet [<User: minj>, <User: s.palazzo-corner19@imperial.ac.uk>, <User: galm>, <User: jletro01@student.bbk.ac.uk>, <User: j.rogelj@imperial.ac.uk>]>

In [87]:
print(r.url)
html_doc = r.text
soup = BeautifulSoup(html_doc, 'html.parser')
soup.prettify()

https://linkinghub.elsevier.com/retrieve/pii/S0378778821009567


'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "https://www.w3.org/TR/html4/loose.dtd">\n<html>\n <head>\n  <meta charset="utf-8"/>\n  <meta content="text/html" http-equiv="Content-Type"/>\n  <meta content="2; url=\'/retrieve/articleSelectSinglePerm?Redirect=https%3A%2F%2Fwww.sciencedirect.com%2Fscience%2Farticle%2Fpii%2FS0378778821009567%3Fvia%253Dihub&amp;key=c59e2457a94e37e3cc240380f6e1a0834386a377\'" http-equiv="REFRESH">\n   <script type="text/javascript">\n    (window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam-cell.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"ee15051b1b",applicationID:"81847441"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}functio

In [28]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
}
res = requests.get(row['wosarticle__di'], headers=headers)

In [29]:
res

<Response [503]>